# Exponential Disc

In [1]:
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
from pyDOE import lhs

sys.path.append('../')
from gpinn.network import FCN
from gpinn.training import TrainingPhase

plt.style.use('seaborn-v0_8-darkgrid')

As a reminder, the equation we want to solve is the following :

$$ \dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi'}{\partial R'}\right) + \dfrac{1}{\eta^{2}}\dfrac{\partial^2 \Phi'}{\partial z'^2} = e^{-R'} \cosh^{-2}{z'}$$

where we have defined : $\phi'= \frac{\phi}{G M_d/z_d}$, $z' = \dfrac{z}{z_d}$, $R' = \dfrac{R}{R_d}$and $\eta = \frac{z_d}{R_d} $.

Therefore the residual would be written as follows:
$$\dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi'}{\partial R'}\right) + \dfrac{1}{\eta^{2}}\dfrac{\partial^2 \Phi'}{\partial z'^2} - e^{-R_i'} \cosh^{-2}{z_i'}$$

and the loss function for the collocation points:
$$ \mathcal{L}_{\mathcal{F}}(\theta) = MSE_{\mathcal{F}} = \dfrac{1}{N_c}\sum_{i=1}^{N_c}\left\|\dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi'}{\partial R'}\right) + \dfrac{1}{\eta^{2}}\dfrac{\partial^2 \Phi'}{\partial z'^2} - e^{-R_i'} \cosh^{-2}{z_i'}\right\|^2$$

$z_d$, for now, we will set it in order to have $\dfrac{z_d}{R_d}=0.2$

The domain where to solve the equations can be between $\dfrac{R}{R_d} = [0;20]$  and $\dfrac{z}{z_d} = [0;5]$

We should also investigate where and when is possible to solve the equation via approximations.

To compare our results, we set here :
- $R_d = 4$  
- $\dfrac{z_d}{R_d}=0.2$  
- $M_d = 10^{10.5}$   

In [2]:
def pde_residual(nn, x_pde):
    eta = 0.2
    r, z = x_pde[:, 0].unsqueeze(1), x_pde[:, 1].unsqueeze(1)
    x_pde.requires_grad = True 
    f = nn(x_pde)
    # -------- Differentiation w.r.t. R ----------------
    f_r = torch.autograd.grad(f, r, torch.ones(x_pde.shape[0], 1),retain_graph=True, create_graph=True)[0]
    f_r = f_r[:, 0].unsqueeze(1)
    f_r = f_r * r
    f_rr = torch.autograd.grad(f_r, r, torch.ones(x_pde.shape[0], 1), retain_graph=True, create_graph=True)[0]
    # -------- Differentiation w.r.t. z ----------------
    f_z = torch.autograd.grad(f, z, torch.ones(x_pde.shape[0], 1), retain_graph=True, create_graph=True)[0]
    f_zz = torch.autograd.grad(f_z, z, torch.ones(x_pde.shape[0], 1), retain_graph=True, create_graph=True)[0]
    lhs = 1/r * f_rr + 1/eta**2 * f_zz
    rhs = torch.exp(r) * torch.cosh(z)**(-2)
    return lhs - rhs

In [3]:
def mse(residual: torch.Tensor):
    return residual.pow(2).mean()

def rmse(residual: torch.Tensor):
    return torch.sqrt(residual.pow(2).mean())

def mae(array: torch.Tensor):
    return torch.abs(array).mean()

## Parameters

In [4]:
steps = 10_000

layers = np.array([2, 32, 16, 1])

# To generate new data:
z_min = 0
z_max = 5
r_min = 0
r_max = 20
total_points_z = 150
total_points_r = 150
# Nu: Number of training points # Nf: Number of collocation points (Evaluate PDE)
Nu = 100
Nf = 1024

## Data Generation

In [5]:
z = torch.linspace(z_min, z_max, total_points_z).view(-1, 1)
r = torch.linspace(r_min, r_max, total_points_r).view(-1, 1)
# Create the mesh
R, Z = torch.meshgrid(r.squeeze(1), z.squeeze(1), indexing='xy')
# y_real = dehnen(X, GAMMA)

## Boundary Conditions

### Left Boundary

Corresponds to $R'=0$ and $z' \in [0, 5]$.

In [ ]:
left_X = torch.hstack((R[:, 0][:, None], Z[:, 0][:, None]))
left_Y = dehnen(left_X[:, 0], GAMMA[:, 0]).unsqueeze(1)

### Bottom Boundary 

Corresponds to $R'\in [0, 20]$ and $z' = 0$.

In [ ]:
bottom_X = torch.hstack((X[0, :][:, None], GAMMA[0, :][:, None]))
bottom_Y = dehnen(bottom_X[:, 0], GAMMA[0, :]).unsqueeze(1)

### Top Boundary

Corresponds to $R'\in [0, 20]$ and $z' = 5$.

In [ ]:
# ------------------------- Upper Boundary -------------------------
top_X = torch.hstack((X[-1, :][:, None], GAMMA[-1, :][:, None]))
top_Y = dehnen(top_X[:, 0], GAMMA[-1, :]).unsqueeze(1)

### Right Boundary

Corresponds to $R'=20$ and $z' \in [0, 5]$.

In [ ]:
# ------------------------- Right Boundary -------------------------
right_X = torch.hstack((X[:, -1][:, None], GAMMA[:, -1][:, None]))
right_Y = dehnen(right_X[:, 0], GAMMA[:, 0]).unsqueeze(1)

### Testing Data

In [ ]:
# Transform the mesh into a 2-column vector
x_test = torch.hstack((X.transpose(1, 0).flatten()[:, None], GAMMA.transpose(1, 0).flatten()[:, None]))
y_test = y_real.transpose(1, 0).flatten()[:, None]  # Colum major Flatten (so we transpose it)
# Domain bounds
lb = x_test[0]  # first value
ub = x_test[-1]  # last value

### Training Data

In [ ]:
X_train = torch.vstack([left_X, bottom_X, top_X, right_X])
Y_train = torch.vstack([left_Y, bottom_Y, top_Y, right_Y])
# Choose(Nu) points of our available training data:
idx = np.random.choice(X_train.shape[0], Nu, replace=False)
X_train_Nu = X_train  # [idx, :]
Y_train_Nu = Y_train  # [idx, :]
# Collocation Points (Evaluate our PDe)


# Choose(Nf) points(Latin hypercube)
X_train_Nf = lb + (ub - lb) * lhs(2, Nf)  # 2 as the inputs are x and gamma
X_train_Nf = torch.vstack((X_train_Nf, X_train_Nu))  # Add the training points to the collocation point

### Initial Points

In [ ]:
plt.figure()
plt.title("Configuration of training points")
plt.plot(X_train_Nu[:, 0], X_train_Nu[:, 1], 'xr', label="Boundary Points")
plt.plot(X_train_Nf[:, 0], X_train_Nf[:, 1], '.b', label="Collocation Points")
plt.xlabel('$s$')
plt.ylabel(r'$\gamma$')
plt.legend()
plt.show()
plt.close()

### Training

In [ ]:
X_test = x_test.float()  # the input dataset (complete)
Y_test = y_test.float()  # the real solution

# Create Model
PINN = FCN(layers) #, act=torch.nn.SiLU())

print(PINN)

training = TrainingPhase(neural_net=PINN, training_points=(X_train_Nu, Y_train_Nu, X_train_Nf),
                         testing_points=(X_test, Y_test), equation=pde_residual, n_epochs=steps,
                         optimizer=torch.optim.Adam,
                         _loss_function=rmse)

net, epochs, losses = training.train_model()

In [ ]:
np.save("../arrays/loss_disc.npy", losses)
np.save("../arrays/epochs_disc.npy", epochs)
training.save_model("../models/exp_disc.pt")

### Comparing Results

In [ ]:
data = np.loadtxt("test_phi_grid.dat")
R_test, z_test, phi_test = data.T

R_test = R_test.reshape(250, 250)
z_test = z_test.reshape(250, 250)
phi_test = phi_test.reshape(250, 250)

In [ ]:
Md = 10**(10.5)

In [ ]:
input_matrix = np.array([R_test.flatten(), z_test.flatten()]).T
phi_ = PINN(input_matrix)

In [ ]:
plt.figure()
plt.title("Relative Error")
plt.pcolormesh(R_test, z_test, np.abs(phi_test - phi_)/phi_)
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\dfrac{\Delta \Phi(R, z)}{\Phi(R, z)}$");